In [130]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import pathlib
import re
import datetime
#import gensim
#import spacy

In [2]:
wd = '/home/beeb/Insync/sei2112@columbia.edu/Google Drive/Columbia SusDev PhD/Research/My work/parler'
os.chdir(wd)

In [3]:
filepath = 'data/raw/parler_2020-01-06_posts-partial/files_f/f000edcac32f48f3aafdeaa9aad1a113'
with open(filepath, 'r') as f:
    dat = f.read()
soup = BeautifulSoup(dat)

## STEPS TO TAKE
- extract user name
- extract text
- extract timestamp

In [7]:
soup = BeautifulSoup(dat)

In [16]:
text = [i.get_text() for i in soup.find_all('div', {'class': 'card--body'})]

In [18]:
author = soup.find_all('span', {'class': 'author--username'})

In [20]:
def parse(section, bit):
    return [i.get_text() for i in soup.find_all(section, {'class': bit})]

In [127]:
text = parse('div', 'card--body')
author = parse('span', 'author--username')
timestamp = parse('span', 'post--timestamp')


In [115]:
def extractInfo(link):
    l = link.find('img')
    if l is None:
        return False
    else:
        return link.find('img')['alt'] == 'Post Comments'

In [104]:

interaction = soup.find(extractInfo).find_all('span',{'class': 'pa--item--count'})
comments = interaction[0].get_text()
echoes = interaction[1].get_text()
upvotes = interaction[2].get_text()

In [144]:
def getData(filepath, outpath = 'data/intermediate/parlerText/inParts/{}'):
    fname = pathlib.Path(filepath)
    createTime = datetime.datetime.fromtimestamp(fname.stat().st_ctime)
    with open(filepath, 'r') as f:
        dat = f.read()
    soup = BeautifulSoup(dat)
    text = parse('div', 'card--body')
    author = parse('span', 'author--username')
    timestamp = parse('span', 'post--timestamp')
    interaction = soup.find(extractInfo).find_all('span',{'class': 'pa--item--count'})
    comments = interaction[0].get_text()
    echoes = interaction[1].get_text()
    upvotes = interaction[2].get_text()
    # unclear why there are multiple bits of text/author per page here, but let's just
    # select the first because that seems to be the important one?
    fileID = filepath.split('/')[-1]
    res = pd.DataFrame(index =  [fileID], data = {'text': text[0], 'author': author[0], 
                        'timestamp': timestamp[0], 'createTime': createTime,
                       'comments': comments, 'echoes': echoes, 'upvotes': upvotes})
    res.to_csv(outpath.format(fileID))
    